In [17]:
%matplotlib inline

In [18]:
# Reguired libraries
import os
import pandas as pd
import xray
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.basemap import cm as basemap_cm
import seaborn as sns
from collections import OrderedDict
from netCDF4 import num2date
from scipy.stats import ranksums, ttest_ind
from scipy.spatial import ConvexHull
from datetime import datetime
from calendar import month_abbr, month_name
from netCDF4 import date2num, num2date

# For temporary display of existing figures
from IPython.display import Image

# RASM lib plotting utilities
from rasmlib.calendar import dpm
from rasmlib.analysis.climatology import season_mean, annual_mean
from rasmlib.analysis.plotting import cmap_discretize, sub_plot_pcolor, projections, default_map, make_bmap, seasons

# Set some general plotting values
fontsize = 7
dpi = 200 # set to 900 for final publication
mpl.rc('font', family='sans-serif') 
mpl.rc('font', serif='Myriad Pro') 
mpl.rc('text', usetex='false') 
mpl.rcParams.update({'font.size': fontsize})
mpl.rcParams['mathtext.default'] = 'sf'
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['pdf.fonttype'] = 42

fill_color = (0.9, 0.9, 0.9)

In [19]:
ncfiles = {}

# Grid and domain files
ncfiles['rasm_domain'] = '/raid2/jhamman/projects/RASM/data/inputdata/CESM/share/domains/domain.lnd.wr50a_ar9v4.100920.nc'
ncfiles['rasm_lnd_masks'] = '/raid2/jhamman/projects/RASM/data/inputdata/RASM_VICRVIC_GRID_MASKS_AND_METRICS.nc'
ncfiles['rasm_ocn_masks'] = '/raid2/jhamman/projects/RASM/data/inputdata/RASM_WRFVIC_GRID_MASKS_AND_METRICS.nc'

rasm_sims = ['rasm_cpl_era_monthly_ts', 'rasm_cpl_cfsr_monthly_ts']
ncfiles['rasm_cpl_era_monthly_ts'] = '/raid2/jhamman/projects/RASM/data/processed/R1009RBRceap01a/cpl/monthly_mean_timeseries/R1009RBRceap01a.cpl.hmm.197909-201412.nc'
ncfiles['rasm_cpl_cfsr_monthly_ts'] = '/raid2/jhamman/projects/RASM/data/processed/R1009RBRceap01b/cpl/monthly_mean_timeseries/R1009RBRceap01b.cpl.hmm.197909-200912.nc'
ncfiles['era_monthly_ts3'] = '/raid2/jhamman/projects/RASM/data/compare/era-interim/era_water_vars.1979-2014.nc'
ncfiles['merra_monthly_ts1'] = '/raid2/jhamman/projects/RASM/data/compare/MERRA/monthly/MERRA.prod.assim.tavgM_2d_lnd_Nx.197901-201312.SUB.wr50a.nc'
ncfiles['cru_monthly_ts'] = '/raid2/jhamman/projects/RASM/data/compare/cru_ts3.21/cru_ts3.21.1901.2012_wr50a.nc'
ncfiles['sheffield_monthly_ts'] = '/raid2/jhamman/projects/RASM/data/compare/sheffield2006/sheffield2006_wr50a.mm.nc'
ncfiles['adam_monthly_ts'] = '/raid2/jhamman/projects/RASM/data/compare/adam2003/adam2003_wr50a.mm.nc'

start='1990-01-01'
end='2014-12-31'

ncdata = {}
for k, v in ncfiles.items():
    try:
        print(k)
        ncdata[k] = xray.open_dataset(v)
    except:
        print(k, '<---unable to decode time!', )

# Rename variables in datasets as necessary
name_dicts = {'era_monthly_ts3': {'tp': 'Precipitation',
                                  'e': 'Evap',
                                  'ro': 'Runoff'},
              'sheffield_monthly_ts':{'prcp': 'Precipitation'},
              'merra_monthly_ts1': {'prectot': 'Precipitation',
                                    'runoff': 'Runoff',
                                    'baseflow': 'Baseflow',
                                    'shland': 'Senht',
                                    'lhland': 'Latht',
                                    'snomas': 'Swq',
                                    'evland': 'Evap',
                                    'lwland': 'Lwnet',
                                    'swland': 'Swnet'},
              'cru_monthly_ts': {'pre': 'Precipitation'},
              'adam_monthly_ts': {'Precip': 'Precipitation'},
              }

for k, v in ncdata.items():
    if k in name_dicts:
        print(k)
        v.rename(name_dicts[k], inplace=True)
        

# Setup rasmlib plotting
wr50a_map = make_bmap(projection=projections['wr50a'],
                      lons=ncdata['rasm_domain']['xc'].values,
                      lats=ncdata['rasm_domain']['yc'].values)

adam_monthly_ts
rasm_domain
era_monthly_ts3
merra_monthly_ts1
cru_monthly_ts
rasm_cpl_cfsr_monthly_ts
rasm_ocn_masks
rasm_cpl_era_monthly_ts
sheffield_monthly_ts
rasm_lnd_masks
adam_monthly_ts
era_monthly_ts3
merra_monthly_ts1
cru_monthly_ts
sheffield_monthly_ts


In [20]:
lnd_mask_keys = ['mask_rof_to_centralarctic', 'mask_rof_to_kara', 'mask_rof_to_barents']

lnd_mask = np.zeros_like(ncdata['rasm_lnd_masks'][lnd_mask_keys[0]].values)
for k in lnd_mask_keys:
    lnd_mask += ncdata['rasm_lnd_masks'][k].values
lnd_mask = lnd_mask.clip(0., 1.)

ocn_mask_keys = ['mask_centralarctic', 'mask_kara', 'mask_barents']

ocn_mask = np.zeros_like(ncdata['rasm_ocn_masks'][ocn_mask_keys[0]].values)
for k in ocn_mask_keys:
    ocn_mask += ncdata['rasm_ocn_masks'][k].values
ocn_mask = ocn_mask.clip(0., 1.)

full_mask = lnd_mask + ocn_mask
full_mask = full_mask.clip(0., 1.)

all_ones_mask = np.ones_like(full_mask)
else_mask = all_ones_mask - full_mask


In [21]:
re = 6.37122e6
rho = 1.000e3
days_per_year = 365.
seconds_per_year =days_per_year * 86400.
mm_per_m = 1000.
m_per_km = 1000.
m3_to_km3 = 1 / m_per_km**3
kg_to_km3 = m3_to_km3 / rho

In [22]:
# Add variables or adjust units of datasets if ncessary

# RASM
re = 6.37122e6
ncdata['rasm_domain']['area'] *= re * re  # m2

for sim in rasm_sims:
    ncdata[sim]['Precipitation'] = ncdata[sim]['a2xavg_Faxa_rainc'] + ncdata[sim]['a2xavg_Faxa_rainl'] + ncdata[sim]['a2xavg_Faxa_snowc'] + ncdata[sim]['a2xavg_Faxa_snowl']

# ERA-Interim
for var in ('Runoff', 'Precipitation'):
    ncdata['era_monthly_ts3'][var] *= 1000.
ncdata['era_monthly_ts3']['Evap'] *= -1000.
ncdata['era_monthly_ts3'] = ncdata['era_monthly_ts3'].resample('MS', how='mean', dim='time')
ncdata['era_monthly_ts3']['runoff_tot'] = ncdata['era_monthly_ts3']['Runoff']
ncdata['era_monthly_ts3']['P-E'] = ncdata['era_monthly_ts3']['Precipitation'] - ncdata['era_monthly_ts3']['Evap']
ncdata['era_monthly_ts3'].drop(['nv4', 'xc_bnds', 'yc_bnds'])


# MERRA
ncdata['merra_monthly_ts1']['Precipitation'] *= 86400 
ncdata['merra_monthly_ts1']['Runoff'] *= 86400 
ncdata['merra_monthly_ts1']['Baseflow'] *= 86400 
ncdata['merra_monthly_ts1']['Evap'] *= 86400 
ncdata['merra_monthly_ts1']['runoff_tot'] = ncdata['merra_monthly_ts1']['Runoff'] + ncdata['merra_monthly_ts1']['Baseflow']
ncdata['merra_monthly_ts1']['runoff_ratio'] = ncdata['merra_monthly_ts1']['runoff_tot'] / ncdata['merra_monthly_ts1']['Precipitation']
ncdata['merra_monthly_ts1']['P-E'] = ncdata['merra_monthly_ts1']['Precipitation'] - ncdata['merra_monthly_ts1']['Evap']

In [23]:
def mask_to_vol(var, mask, domain, units='kg m-2 s-1', days_per_year=365):
    '''
    var (units: kg m-2 s-1) and mask are numpy arrays, domain is a xray dataset
    
    returns volume in km3
    '''
    if units == 'kg m-2 s-1':
        units_mult = kg_to_km3 * days_per_year * 86400
    elif units == 'mm d-1':
        units_mult = days_per_year * m3_to_km3 / mm_per_m
    
    ys, xs = np.nonzero(mask)
    return (var[ys, xs] * domain['area'].values[ys, xs] * mask[ys, xs]).sum() * units_mult
    

In [24]:
def empty_df():
    df = pd.DataFrame()
    df['P_O'] = [np.nan]
    df['E_O'] = [np.nan]
#     df['Inflow_O'] = np.nan
#     df['Outflow_O'] = np.nan
    df['P_L'] = [np.nan]
    df['ET_L'] = [np.nan]
    df['R_L'] = [np.nan]
    df['C_A'] = [np.nan]
    
    return df

def get_summary(data, var, mask, **kwargs):
    
    try:
        return mask_to_vol(data[var].values, mask, ncdata['rasm_domain'], **kwargs)
    except:
        return np.nan
    
    return val

In [25]:
# summary['rasm_cpl_era_monthly_ts_wrfphys']['P_L'] - summary['rasm_cpl_era_monthly_ts_wrfphys']['R_L'] - summary['rasm_cpl_era_monthly_ts_wrfphys']['ET_L']

In [26]:
#  domain mask
summary = OrderedDict()
for sim in rasm_sims:
    print(sim)
    summary[sim] = empty_df()
    try:
        ds = ncdata[sim][['Precipitation', 'l2xavg_Flrl_rofliq', 'l2xavg_Fall_evap', 'x2aavg_Faxx_evap']]
    except:
        ds = ncdata[sim][['Precipitation', 'l2xavg_Flrl_rofliq']]
   
    cpl_annual_mean = annual_mean(ds.sel(time=slice(start, end)), calendar='noleap')

    summary[sim]['P_L'][0] = get_summary(cpl_annual_mean, 'Precipitation', all_ones_mask)
    summary[sim]['ET_L'][0] = -1 * get_summary(cpl_annual_mean, 'x2aavg_Faxx_evap', all_ones_mask)

    try:
        cpl_annual_mean['C_A'] = cpl_annual_mean['Precipitation']
        cpl_annual_mean['C_A'] += cpl_annual_mean['x2aavg_Faxx_evap'].values
        summary[sim]['C_A'][0] = get_summary(cpl_annual_mean, 'C_A', all_ones_mask)
    except:
        summary[sim]['C_A'][0] = np.nan
        
    print(summary[sim])

rasm_cpl_era_monthly_ts
   P_O  E_O           P_L          ET_L  R_L           C_A
0  NaN  NaN  92875.694836  72767.580519  NaN  20108.114316
rasm_cpl_cfsr_monthly_ts
   P_O  E_O           P_L          ET_L  R_L           C_A
0  NaN  NaN  95453.133738  72678.291192  NaN  22774.842546


In [27]:
#  else mask
summary = OrderedDict()
for sim in rasm_sims:
    print(sim)
    summary[sim] = empty_df()
    try:
        ds = ncdata[sim][['Precipitation', 'l2xavg_Flrl_rofliq', 'l2xavg_Fall_evap', 'x2aavg_Faxx_evap']]
    except:
        ds = ncdata[sim][['Precipitation', 'l2xavg_Flrl_rofliq']]
   
    cpl_annual_mean = annual_mean(ds.sel(time=slice(start, end)), calendar='noleap')

    summary[sim]['P_L'][0] = get_summary(cpl_annual_mean, 'Precipitation', else_mask)
    summary[sim]['ET_L'][0] = -1 * get_summary(cpl_annual_mean, 'x2aavg_Faxx_evap', else_mask)

    try:
        cpl_annual_mean['C_A'] = cpl_annual_mean['Precipitation']
        cpl_annual_mean['C_A'] += cpl_annual_mean['x2aavg_Faxx_evap'].values
        summary[sim]['C_A'][0] = get_summary(cpl_annual_mean, 'C_A', else_mask)
    except:
        summary[sim]['C_A'][0] = np.nan
        
    print(summary[sim])

rasm_cpl_era_monthly_ts
   P_O  E_O           P_L          ET_L  R_L           C_A
0  NaN  NaN  81885.562632  66927.180987  NaN  14958.381645
rasm_cpl_cfsr_monthly_ts
   P_O  E_O           P_L          ET_L  R_L           C_A
0  NaN  NaN  84473.616716  66846.183967  NaN  17627.432749


In [28]:
# land / ocean masks
summary = OrderedDict()
for sim in rasm_sims:
    print(sim)
    summary[sim] = empty_df()
    try:
        ds = ncdata[sim][['Precipitation', 'l2xavg_Flrl_rofliq', 'l2xavg_Fall_evap', 'x2aavg_Faxx_evap']]
    except:
        ds = ncdata[sim][['Precipitation', 'l2xavg_Flrl_rofliq']]
   
    cpl_annual_mean = annual_mean(ds.sel(time=slice(start, end)), calendar='noleap')

    summary[sim]['P_L'][0] = get_summary(cpl_annual_mean, 'Precipitation', lnd_mask)
    summary[sim]['ET_L'][0] = -1 * get_summary(cpl_annual_mean, 'x2aavg_Faxx_evap', lnd_mask)
    summary[sim]['R_L'][0] = get_summary(cpl_annual_mean, 'l2xavg_Flrl_rofliq', lnd_mask)
    summary[sim]['P_O'][0] = get_summary(cpl_annual_mean, 'Precipitation', ocn_mask)
    summary[sim]['E_O'][0] = -1 * get_summary(cpl_annual_mean, 'x2aavg_Faxx_evap', ocn_mask)

    try:
        cpl_annual_mean['C_A'] = cpl_annual_mean['Precipitation']
        cpl_annual_mean['C_A'] += cpl_annual_mean['x2aavg_Faxx_evap'].values
        summary[sim]['C_A'][0] = get_summary(cpl_annual_mean, 'C_A', full_mask)
    except:
        summary[sim]['C_A'][0] = np.nan
        
    print(summary[sim])

rasm_cpl_era_monthly_ts
           P_O         E_O          P_L         ET_L          R_L          C_A
0  2644.372842  569.293984  8346.073802  5271.203182  3072.081764  5149.732672
rasm_cpl_cfsr_monthly_ts
           P_O         E_O         P_L        ET_L          R_L          C_A
0  2626.158716  567.374503  8353.67382  5264.82958  3089.897593  5147.409797


In [31]:
for sim in rasm_sims:
    print(sim, summary[sim]['P_L'] - summary[sim]['ET_L'] - summary[sim]['R_L'])

rasm_cpl_era_monthly_ts 0    2.788855
dtype: float64
rasm_cpl_cfsr_monthly_ts 0   -1.053353
dtype: float64


In [29]:
sim = 'ERA'
print(sim)
summary[sim] = empty_df()

kwargs = dict(units='mm d-1', days_per_year=365.25)

era_annual_mean = annual_mean(ncdata['era_monthly_ts3'].sel(time=slice(start, end)), calendar='standard')

summary[sim]['P_L'][0] = get_summary(era_annual_mean, 'Precipitation', lnd_mask, **kwargs)
summary[sim]['ET_L'][0] = get_summary(era_annual_mean, 'Evap', lnd_mask, **kwargs)
summary[sim]['R_L'][0] = get_summary(era_annual_mean, 'runoff_tot', lnd_mask, **kwargs)
summary[sim]['P_O'][0] = get_summary(era_annual_mean, 'Precipitation', ocn_mask, **kwargs)
summary[sim]['E_O'][0] = get_summary(era_annual_mean, 'Evap', ocn_mask, **kwargs)

try:
    era_annual_mean['C_A'] = era_annual_mean['Precipitation']
    era_annual_mean['C_A'] -= era_annual_mean['Evap'].values
    summary[sim]['C_A'][0] = get_summary(era_annual_mean, 'C_A', full_mask, **kwargs)
except:
    summary[sim]['C_A'][0] = np.nan

summary[sim]

ERA


,P_O,E_O,P_L,ET_L,R_L,C_A
0,2901.19921,1169.346319,7811.97573,4813.036173,3741.581151,4730.599378


In [30]:
sim = 'MERRA'
print(sim)
summary[sim] = empty_df()

kwargs = dict(units='mm d-1', days_per_year=365.25)

merra_annual_mean = annual_mean(ncdata['merra_monthly_ts1'].sel(time=slice(start, end)), calendar='standard')

summary[sim]['P_L'][0] = get_summary(merra_annual_mean, 'Precipitation', lnd_mask, **kwargs)
summary[sim]['ET_L'][0] = get_summary(merra_annual_mean, 'Evap', lnd_mask, **kwargs)
summary[sim]['R_L'][0] = get_summary(merra_annual_mean, 'runoff_tot', lnd_mask, **kwargs)
summary[sim]['P_O'][0] = get_summary(merra_annual_mean, 'Precipitation', ocn_mask, **kwargs)
summary[sim]['E_O'][0] = get_summary(merra_annual_mean, 'Evap', ocn_mask, **kwargs)

try:
    merra_annual_mean['C_A'] = merra_annual_mean['Precipitation']
    merra_annual_mean['C_A'] -= merra_annual_mean['Evap'].values
    summary[sim]['C_A'][0] = get_summary(merra_annual_mean, 'C_A', full_mask, **kwargs)
except:
    summary[sim]['C_A'][0] = np.nan

summary[sim]


MERRA


KeyError: 'cannot represent labeled-based slice indexer for dimension %r with a slice over integer positions; the index is unsorted or non-unique'

In [ ]:
sim = 'ADAM'
print(sim)
summary[sim] = empty_df()

kwargs = dict(units='mm d-1', days_per_year=365.25)

adam_annual_mean = annual_mean(ncdata['adam_monthly_ts'].sel(time=slice(start, end)), calendar='standard')

summary[sim]['P_L'][0] = get_summary(adam_annual_mean, 'Precipitation', lnd_mask, **kwargs)


summary[sim]


In [ ]:
sim = 'SHEFFIELD'
print(sim)
summary[sim] = empty_df()

kwargs = dict(units='kg m-2 s-1', days_per_year=365.25)

sheffield_annual_mean = annual_mean(ncdata['sheffield_monthly_ts'].sel(time=slice(start, end)), calendar='standard').squeeze()

summary[sim]['P_L'][0] = get_summary(sheffield_annual_mean, 'Precipitation', lnd_mask, **kwargs)
summary[sim]['P_O'][0] = get_summary(sheffield_annual_mean, 'Precipitation', ocn_mask, **kwargs)

summary[sim]

In [ ]:
summary['OBSERVATIONS'] = empty_df()
summary['OBSERVATIONS']['R_L'][0] = 3200


In [ ]:
index = summary.keys()
df = pd.concat([summary[key] for key in index])
df.index = index
df
df.T.plot(kind='bar')

In [ ]:
rename_vars = {'P_O': 'Precip\n(Ocean)',
               'E_O': 'Evap\n(Ocean)',
               'P_L': 'Precip\n(Land)',
               'ET_L': 'Evap\n(Land)',
               'R_L': 'Runoff\n(Land)',
               'C_A': 'Convergence\n(Land+Ocean)'}

rename_index = {'rasm_cpl_era_monthly_ts': 'RASM (ERA)', 'rasm_cpl_cfsr_monthly_ts': 'RASM (CFSR)'}

df_rename = df.rename(columns=rename_vars, index=rename_index)

df_rename

In [ ]:
df2 = df_rename
df2['Dataset'] = df_rename.index
melt = pd.melt(df2, 'Dataset')

In [ ]:
flatui = ["#000000", "#000000", "#ec3125", "#4092c5", "#58b95b", "#ab66aa", "#f7941e"]

In [ ]:
sns.set_style("whitegrid", {'legend.frameon': True})

sns.stripplot(x="variable", y="value", data=melt,
              hue='Dataset', palette=flatui,
              jitter=True, edgecolor="grey", split=False, size=12)

plt.xlabel('')
plt.ylabel('Volume (km$^3$/yr)')
plt.ylim([0, 9000])
plt.title('Arctic Basin Freshwater Fluxes', fontsize=16)

In [ ]:
sns.set_style("white")
fig, ax = plt.subplots()

wr50a_map.m.drawparallels(np.arange(-80., 81., 20.), linewidth=0.5)
wr50a_map.m.drawmeridians(np.arange(-180., 181., 20.), linewidth=0.5)
wr50a_map.m.drawcoastlines(color='k', linewidth=0.5)
wr50a_map.m.drawmapboundary(fill_color=fill_color)
wr50a_map.m.fillcontinents(color='white', zorder=0)
wr50a_map.m.contourf(wr50a_map.xi, wr50a_map.yi, np.ma.masked_where(lnd_mask <= 0, lnd_mask), colors=["#2ecc71", "#2ecc71"])
wr50a_map.m.contourf(wr50a_map.xi, wr50a_map.yi, np.ma.masked_where(ocn_mask <= 0, ocn_mask), colors=["#34495e", "#34495e"])

# add legend
lines = []
labels = []
for source, color in zip(['Land', 'Ocean'], ["#2ecc71", "#34495e"]):
    lines.append(mlines.Line2D([], [], color=color, label=var, linewidth=30))
    labels.append(source)

leg = fig.legend(handles=lines, labels=labels, fontsize=25, loc=3, bbox_to_anchor=[0.07, 0.04])
# plt.savefig(os.path.join(os.environ['FTP'], 'RASM_land_surface_climate_figures', 'arctic_flux_domain.pdf'), bbox_inches='tight')


In [ ]:
def add_reservoir(center, dims, text, ax=None, fontsize=12):
    if ax is None:
        ax = plt.gca()
        
    x, y = center
    w, h = dims
        
    ax.add_patch(mpl.patches.Rectangle((x - w / 2., y - h / 2.), w, h, facecolor="white"))
    ax.text(x, y, text, fontsize=fontsize, fontweight='bold',
            horizontalalignment='center', verticalalignment='center')
    
    
annotate_kwargs = dict(xycoords='data', textcoords='data')


def arrow_kwargs(size):
    return dict(color="#34495e",
                length_includes_head=True,
                width=size,
                head_width=1.3 * size,
                head_length=1 * size)
    

In [ ]:
df.P_L.

In [ ]:
max_arrow_width = 1

scale = max_arrow_width / df.mean().max()

sizes = df.mean() * scale

print(sizes)

In [ ]:
sns.set_style("whitegrid")

fig = plt.figure()
fig.suptitle('Arctic Basin Freshwater Budget', fontsize=14, fontweight='bold', y=.99)

ax = fig.add_subplot(111)
ax.set_title('(units: km$^3$/year)')

add_reservoir((6, 9), (7, 1.5), 'ATMOSPHERE', ax=ax)
add_reservoir((3.5, 5), (2, 2), 'OCEAN', ax=ax)
add_reservoir((8, 5), (3, 2), 'LAND', ax=ax)

# Atmosphere Fluxes - (P - E)
ax.arrow(11, 9, -1.5, 0, **arrow_kwargs(sizes['C_A']))
ax.text(9.75, 9.6, 'P-E', fontsize=10, fontweight='bold')

# Land Fluxes - (P)
ax.arrow(7, 8.25, 0, -2.25, **arrow_kwargs(sizes['P_L']))
ax.text(7.25, 6.2, 'P$_L$', fontsize=10, fontweight='bold')

df.P_L


# Land Fluxes - (ET)
ax.arrow(9.25, 6, 0, 2.25, **arrow_kwargs(sizes['ET_L']))
ax.text(9.5, 7.9, 'ET$_L$', fontsize=10, fontweight='bold')

# Land Fluxes - (R)
ax.arrow(6.5, 5, -2, 0, **arrow_kwargs(sizes['ET_L']))
ax.text(4.6, 5.4, 'R', fontsize=10, fontweight='bold')

# Ocean Fluxes - (P)
ax.arrow(2.75, 8.25, 0, -2.25, **arrow_kwargs(sizes['P_O']))
ax.text(3, 6.2, 'P$_0$', fontsize=10, fontweight='bold')

# Ocean Fluxes - (E)
ax.arrow(4.25, 6, 0, 2.25, **arrow_kwargs(sizes['E_O']))
ax.text(4.4, 7.9, 'E$_O$', fontsize=10, fontweight='bold')

# Ocean Fluxes - (Inflow)
ax.arrow(0.5, 5, 2, 0, **arrow_kwargs(0.25))

# Ocean Fluxes - (Outflow)
ax.arrow(3.5, 4, 0, -2, **arrow_kwargs(0.25))


ax.axis([-2, 12.5, 0, 10])
# ax.get_xaxis().set_visible(False)
# ax.get_yaxis().set_visible(False)

ax2 = fig.add_axes([0.13, 0.18, 0.25, 0.25])
ax2.set_title('')

wr50a_map.m.drawparallels(np.arange(-80., 81., 20.), linewidth=0.5)
wr50a_map.m.drawmeridians(np.arange(-180., 181., 20.), linewidth=0.5)
wr50a_map.m.drawcoastlines(color='k', linewidth=0.5)
wr50a_map.m.drawmapboundary(fill_color=fill_color)
wr50a_map.m.fillcontinents(color='white', zorder=0)
wr50a_map.m.contourf(wr50a_map.xi, wr50a_map.yi, np.ma.masked_where(lnd_mask <= 0, lnd_mask), colors=["#2ecc71", "#2ecc71"])
wr50a_map.m.contourf(wr50a_map.xi, wr50a_map.yi, np.ma.masked_where(ocn_mask <= 0, ocn_mask), colors=["#34495e", "#34495e"])

# add legend
lines = []
labels = []
for source, color in zip(['Land', 'Ocean'], ["#2ecc71", "#34495e"]):
    lines.append(mlines.Line2D([], [], color=color, label=var, linewidth=8))
    labels.append(source)

leg = fig.legend(handles=lines, labels=labels, fontsize=10, loc=3, ncol=2, bbox_to_anchor=[0.07, 0.06])